---
# **TELECOM CUSTOMER CHURN PREDICTION PROJECT**
### *Author: Priyanka Lakra*
---

# **1. IMPORT LIBRARIES**


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mysql.connector
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, roc_auc_score
)


# **2. CONFIG / CONSTANTS**

In [ ]:

RANDOM_STATE = 42


# **3. LOAD DATA (FROM MySQL)**


In [ ]:

conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='pricass00',      
    database='telco_churn'
)
query = "SELECT * FROM customers"
df = pd.read_sql(query, conn)

print("Initial shape:", df.shape)
display(df.head())


# **4. DATA CLEANING**


In [ ]:

# standardize column names
df.columns = df.columns.str.strip()

# convert TotalCharges to numeric
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# drop exact duplicates
df.drop_duplicates(inplace=True)

# drop rows with nulls 
df.dropna(inplace=True)

# remove identifier column
if 'customerID' in df.columns:
    df.drop('customerID', axis=1, inplace=True)

print("Cleaned shape:", df.shape)


In [ ]:

# --------------------------
# 5. EXPLORATORY DATA ANALYSIS (EDA)
# --------------------------
plt.figure(figsize=(5,4))
sns.countplot(x='Churn', data=df)
plt.title('Customer Churn Distribution')
plt.show()

print(df.describe(include='all').T)

# quick correlation heatmap for numeric features
numeric_df = df.select_dtypes(include=['int64','float64'])
plt.figure(figsize=(10,8))
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap (Numeric Features)')
plt.show()


# **6. ENCODING CATEGORICAL VARIABLES**


In [ ]:

df2 = df.copy()

# binary-like columns explicitly (these will be label encoded)
binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
# remaining categorical columns to one-hot encode
multi_cat_cols = [
    'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines',
    'Contract', 'PaymentMethod'
]

# Label encode binary columns (including target 'Churn')
le = LabelEncoder()
for col in binary_cols:
    if col in df2.columns:
        df2[col] = le.fit_transform(df2[col].astype(str))

# One-hot encode multi-category columns (drop_first avoids dummy trap)
df2 = pd.get_dummies(df2, columns=[c for c in multi_cat_cols if c in df2.columns], drop_first=True)
print("Data shape after encoding:", df2.shape)


# **7. PREPARE FEATURES & SPLIT**


In [ ]:

X = df2.drop('Churn', axis=1)
y = df2['Churn']

# stratify by y to keep churn ratio same in train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=RANDOM_STATE, stratify=y
)

# Standardize numeric features for models that need it (Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# **8. LOGISTIC REGRESSION (LINEAR MODEL)**


In [ ]:

lr = LogisticRegression(max_iter=2000, random_state=RANDOM_STATE)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
y_prob_lr = lr.predict_proba(X_test_scaled)[:, 1]

print("Logistic Regression Results:")
print("Accuracy:", round(accuracy_score(y_test, y_pred_lr), 4))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob_lr), 4))
print(classification_report(y_test, y_pred_lr))


# **9. RANDOM FOREST (NON-LINEAR MODEL)**


In [ ]:

rf = RandomForestClassifier(
    n_estimators=200, random_state=RANDOM_STATE, class_weight='balanced'
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

print("Random Forest Results:")
print("Accuracy:", round(accuracy_score(y_test, y_pred_rf), 4))
print("ROC-AUC:", round(roc_auc_score(y_test, y_prob_rf), 4))
print(classification_report(y_test, y_pred_rf))


# **10. CONFUSION MATRICES**


In [ ]:

plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_test, y_pred_lr), annot=True, fmt='d')
plt.title('Confusion Matrix - Logistic Regression')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_test, y_pred_rf), annot=True, fmt='d')
plt.title('Confusion Matrix - Random Forest')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


# **11. FEATURE IMPORTANCE (RANDOM FOREST)**


In [ ]:

importances = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
plt.figure(figsize=(8,6))
sns.barplot(x=importances.head(10), y=importances.head(10).index)
plt.title('Top 10 Feature Importances (Random Forest)')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()

# Display top 15 features as a table
display(importances.head(15))


# **12. MODEL COMPARISON**


In [ ]:

comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest'],
    'Accuracy': [accuracy_score(y_test, y_pred_lr), accuracy_score(y_test, y_pred_rf)],
    'ROC-AUC': [roc_auc_score(y_test, y_prob_lr), roc_auc_score(y_test, y_prob_rf)]
})
print("\nModel Comparison:\n")
display(comparison)


# **13. ADD CHURN PROBABILITY**


In [ ]:

# Use logistic regression probabilities
df2['Churn_Probability'] = lr.predict_proba(scaler.transform(X))[:, 1]

output_filename = 'cleaned_telco_churn_with_probs.csv'
df2.to_csv(output_filename, index=False)
print(f"Exported file: {output_filename}")


# **14. BUSINESS ESTIMATES: POTENTIAL RETENTION & SAVINGS**


In [ ]:

# compute observed churn rate safely
if df2['Churn'].dtype == 'object':
    # original df2 has 'Yes'/'No'
    churn_counts = df2['Churn'].value_counts(normalize=True)
    observed_churn_rate = 100.0 * churn_counts.get('Yes', churn_counts.get(1, 0.0))
else:
    # if churn encoded as 0/1 in df2
    observed_churn_rate = 100.0 * df2['Churn'].mean()

print(f"Observed Churn Rate: {observed_churn_rate:.2f}%")

# high-risk customers
high_risk_cutoff = 0.70
high_risk_customers = df2[df2['Churn_Probability'] > high_risk_cutoff]
high_risk_rate = len(high_risk_customers) / len(df2) * 100
print(f"High-Risk Customers (> {high_risk_cutoff} prob): {high_risk_rate:.2f}%")

# assume we retain one-third of high-risk customers with interventions
estimated_retention_pct = high_risk_rate * (1/3)
print(f"Estimated Potential Churn Reduction: {estimated_retention_pct:.2f}%")

# revenue savings estimate (monthly)
avg_monthly_revenue = df2['MonthlyCharges'].mean()
total_customers = len(df2)
potential_retained_customers = total_customers * (estimated_retention_pct / 100)
monthly_savings_inr = avg_monthly_revenue * potential_retained_customers

# convert to Lakhs for readability
monthly_savings_lakhs = monthly_savings_inr / 1e5
print(f"Estimated Monthly Savings: ₹{monthly_savings_inr:,.2f} (≈ ₹{monthly_savings_lakhs:.2f} Lakhs)")


# TELCO CUSTOMER CHURN — BUSINESS INSIGHTS & SUMMARY REPORT
---

## 1. Overall Churn Summary

- **Total customers in dataset:** 7032  
- **Churned:** 1869 (**26.58%**)  
- **Retained:** 5163 (**73.42%**)  

> **Note:** Churn means a customer has cancelled their subscription. This churn rate is much higher than the industry average (10–15%), indicating an urgent need for action. I calculated this by dividing the number of churned customers by the total customer count.

---

## 2. Key Customer Demographics and Behaviors

*Below, I inspected the typical values for the major customer features:*

| Feature           | Mean  | Quick Interpretation                   |
|-------------------|-------|----------------------------------------|
| Gender            | 0.50  | Customer base is evenly split          |
| SeniorCitizen     | 0.16  | Majority are not seniors               |
| Partner           | 0.48  | Around half have life partners         |
| Dependents        | 0.30  | 7 out of 10 have no dependents         |
| PhoneService      | 0.90  | Almost all use phone service           |
| PaperlessBilling  | 0.59  | Majority on digital billing            |

- **Average Tenure (months):** 32.4
- **Monthly Charges (mean):** ₹64.8
- **Total Charges (mean):** ₹2283.3

**How to read this:** For instance, “mean 0.30” dependents means 30% of customers have someone else depending on them, while 70% don’t.

---

## 3. What Differs Most—Retained vs Churned Customers?

*Here, I compared the “median” values for key features between customers who stayed and those who left.*

| Feature           | Median Retained | Median Churned | Gap     | Business Clue                       |
|-------------------|-----------------|---------------|---------|-------------------------------------|
| Tenure            | 38              | 10            | +28     | Newer customers churn sooner        |
| MonthlyCharges    | 64.5            | 79.7          | –15.2   | Churners tend to pay higher bills   |
| TotalCharges      | 1683.6          | 703.5         | +980    | Leavers have lower lifetime spend   |

*Smaller or zero gaps can mean the feature has less impact. Here, “Tenure” (how long a customer has stayed) is the strongest driver of churn.*

---

## 4. What Features Best Predict Churn?

| Feature Name        | Correlation with Churn (absolute value) |
|---------------------|-----------------------------------------|
| Churn_Probability   | 0.557                                   |
| Tenure              | 0.354                                   |
| TotalCharges        | 0.199                                   |
| MonthlyCharges      | 0.193                                   |
| PaperlessBilling    | 0.191                                   |

> **Takeaway:** The shorter the tenure and the higher the bill, the more likely a customer will leave.

---

## 5. Where Are the Big Differences? (Percentile Analysis)

*I looked at the 25th percentile (Q1) and 75th percentile (Q3) to understand customer spread:*

- **Tenure:** Q1 = 9 months; Q3 = 55 months  
  Many new customers churn quickly. Focusing on customers in their first year can help.
- **MonthlyCharges:** Q1 = ₹35.59; Q3 = ₹89.86  
  High-bill customers are more at risk.
- **TotalCharges:** Q1 = ₹401; Q3 = ₹3794  
  Customers who pay less overall are at highest risk.

---

## 6. High-Risk Group from Model Predictions

- **High-risk (>0.7 probability):** 6.33% of customers
- These are the most likely to churn, as predicted by the logistic regression/random forest model.

---

## 7. What Drives Churn? (Model-Based Insights)

- Churn is **highest** among:
    - New customers (short tenure)
    - Month-to-month subscriptions
    - Customers with higher bills
    - Customers without tech support
    - Those on “electronic check” payment
    - Fiber optic subscribers (possibly due to service or cost)

> **Interpretation:** Retention should focus on new sign-ups, and offer support/bundling to those at risk.

---

## 8. What I Recommend (If I Were In the Business Team)

| Finding                        | Action Recommended                     |
|---------------------------------|----------------------------------------|
| New customers churn fast        | Improve onboarding, check-in after signup |
| High monthly bills, pay by check| Offer discounts, bundle plans, promote auto-pay |
| Short contracts                 | Encourage switching to long-term plans |
| Lack of support                 | Offer free or discounted tech support  |
| Fiber optic churn               | Review service/price for this group    |

---

## 9. Executive Overview

- This dataset’s churn rate is much higher than what’s considered safe, risking company revenue.
- The main reasons for churn are being new, having a flexible contract, higher bills, or lack of support.
- Models can successfully spot who is at risk, especially those flagged “high probability.”
- If the company acts on these insights, they can likely save both customers and significant revenue.

---

## 10. Business Impact (Estimated)

- **Potential to reduce churn by:** approx. 6%
- **Estimated monthly revenue savings:** ₹20 Lakhs
- **Projected annual benefit:** ₹2.4–3 crore

> Investing in targeted retention (focusing especially on high-risk and high-value customers) is likely to have the maximum business impact.

---

